1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
  
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서로 LLM에 질문과 같이 전달

# 1. 문서의 내용을 읽는다.

In [1]:
%pip install python-docx

     -------------------------------------- 253.0/253.0 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 4.0/4.0 MB 23.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from docx import Document

document = Document("./tax.docx")
print(f'document == {document}')
full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph {index} == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

# 2. 문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [11]:
import tiktoken

def split_text(full_text, chunk_size) :
  encoder = tiktoken.encoding_for_model("gpt-4o")
  total_encoding = encoder.encode(full_text)
  total_token_count = len(total_encoding)
  text_list = []
  for i in range(0, total_token_count, chunk_size) :
    chunk = total_encoding[i:i + chunk_size]
    decoded = encoder.decode(chunk)
    text_list.append(decoded)
  
  return text_list

In [12]:
chunk_list = split_text(full_text, 1500)

# 3. 문서 임베딩

In [ ]:
%pip install chromadb

In [26]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
open_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(
  api_key=open_api_key,
  model_name='text_embedding-3-large')

In [34]:
collection_name = 'tax_collection2'
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [35]:
id_list = []
for index in range(len(chunk_list)) :
  id_list.append(f'tax_doc_chunk_{index}')

In [36]:
len(id_list) == len(chunk_list)

True

In [38]:
tax_collection.add(documents=chunk_list, ids=id_list)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `text_embedding-3-small` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}} in add.